__Preambule__

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%%capture
from tqdm.notebook import tqdm
tqdm().pandas()

In [3]:
## importing stuff

import json
from collections import OrderedDict
from copy import deepcopy

import os
import pickle
from time import sleep
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
import javabridge
from bioformats import JARS as bfJARS
javabridge.start_vm(class_path=bfJARS, max_heap_size="20G")
# import numpy as np
# import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2
from IPython.display import display
import pandas as pd
from islets.Recording import Recording, parse_leica
from islets.utils import get_filterSizes
from general_functions import td_nanfloor, td2str
from copy import deepcopy

from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html

def cellTransform(cellvalue, id=None, verbose=False):
    import numpy as np
    
    if issubclass(type(cellvalue),(bool,np.bool_)):
        if verbose:
            print ("creating a checklist")
        out = dcc.Checklist(id=id,
                             options=[{"label":"✔" if cellvalue else "✘","value":None}],
                             value=[None]*(1-int(cellvalue)),
                             labelStyle={
                                 "color":"green" if cellvalue else "red",
#                                  "padding":"3px"
                             }
                            )
#     elif issubclass(type(cellvalue),(dict,)):
#         out = dcc.Checklist(id=id,
#                              options=[{"label":(str(k).replace(" ","")+("✔" if cellvalue[k] else "✘")), "value":k} for k in cellvalue],
#                              value = [k for k in cellvalue if not cellvalue[k]],
# #                              style={"height":"10px",},
#                              labelStyle={
#                                  "width":"100px",
#                                  "display":"block"
#                                         }
#                             )
    elif issubclass(type(cellvalue),(str,int,float,np.int_)):
        out = cellvalue
    else:
        out = str(cellvalue)
    return html.Div(out,style={
                                 "font-family":"monospace",
                                 "font-size":"14px",
                             },)
#     if issubclass(type(value),str):

def mystyle(col):
    return {"border":'thin lightgrey solid',
            "text-align": "left" if "pickle" in col else None,
            "width": {
                "Series Durations":"120px",
                "Duration":"100px",
                "pickles":"150px",
                "exp":"150px",
                "series":"150px",
                     }.get(col),
           }

def generate_table(dataframe, max_rows=100):
    rows = []
    for i in range(min(len(dataframe), max_rows)):
        row = []
        for col in dataframe.columns:
            value = dataframe.iloc[i][col]
#             row.append(html.Td(value if issubclass(type(value),str) else dcc.Checklist(options=[{"label":"item"}]),
# #                                style=style
#                               ))
            row.append(html.Td(cellTransform(value, id=f"table-{col}-{i}"),
                               style=mystyle(col),
                              ))
        rows.append(html.Tr(row,
                            style={"border":'thin lightgrey solid'},
#                             border='thin lightgrey solid' 
                           ))

    return html.Table(
        # Header
        [html.Tr([html.Th(col, style=mystyle(col)) for col in dataframe.columns],
                            style={"border":'thick lightgrey solid'},)] +
        # Body
        rows,
        style={
            "width": "1300px",
            "text-align":"right",
            "table-layout": "fixed"
        }
    ) 

In [4]:
def import_data(recordings,forceMetadataParse=False):
    status = []
    ilifs = 0
    for pathToRecording in tqdm(recordings):
        print ("#"*20, pathToRecording)
        try:
            rec = Recording(pathToRecording)
            if forceMetadataParse:
                rec.parse_metadata()
                rec.save_metadata()
        except:
            continue
        recType = "Nikon" if pathToRecording.endswith(".nd2") else "Leica"

        if recType=="Leica":
            sers = parse_leica(rec)
        else:
            sers = ["all"]

        analysisFolder = os.path.join(rec.folder, rec.Experiment+"_analysis")
    #     if not os.path.isdir(analysisFolder):
    #         os.makedirs(analysisFolder)

        for ser in sers:
            md = pd.Series(dtype=object)
            md["path"] = pathToRecording
            md["exp"] = os.path.split(pathToRecording)[-1]
            md["series"] = ser
            ser0 = "-".join(ser.split("-")[:-1]) if "-" in ser else ser
            md["line scan"] = rec.metadata.query(f"Name=='{ser0}'").iloc[0].SizeY>3000
            try:
                rec.import_series(ser, onlyMeta=True, isLineScan=md["line scan"])
            except:
                print (f"could not import {ser}")
                status += [md]
                continue
            for k,v in rec.Series[ser]["metadata"].items():
                md[k] = v
                
            saveDir = os.path.join(analysisFolder, ser)
            for k in ["bit depth", "Start time", "End time","Name"]: # , "individual Series"
                try:    del md[k]
                except: pass
            md["Series Durations"] = ["%s [%s]"%(r["Name"].replace("Series","S"), td2str(td_nanfloor(r["Duration"]))) for _,r in md["individual Series"].iterrows()]
            md["Series Durations"] = "\n".join(md["Series Durations"])
            del md["individual Series"]
            md["line scan"] = md["SizeY"]*md["pxSize"]<3 # treat also this as a line scan
            if not md["line scan"]:
                fs = get_filterSizes(md.pxSize)
                movieFilename = os.path.join(saveDir, rec.Experiment+"_"+ser+".mp4")
                md["movie"] = os.path.isfile(movieFilename)
                if md["movie"]:
                    md["movie size [MB]"] = os.path.getsize(movieFilename)/10**6
                pklsDone = {}
                imgsDone = {}
                for fsize in fs:
                    pickleFile = os.path.join(saveDir, ".".join(map(str,fsize))+"_rois.pkl")
                    imageFile = os.path.join(saveDir, ".image_"+".".join(map(str,fsize))+".png")
                    pickleThere = os.path.isfile(pickleFile)
                    imageThere = os.path.isfile(imageFile)
                    pklsDone[fsize] = pickleThere
                    imgsDone[fsize] = imageThere
                md["pickles"] = [pk if len(pk)>1 else pk[0] for pk in pklsDone.keys()]
                md["(re)do pickles"] = all(pklsDone.values())
            status += [dict(md.items())]
        ilifs +=1
    #     if ilifs>3:
    #         break

    return pd.DataFrame(status)

Enter Path to the folder you wish to process:

In [5]:
# mainFolder = "/data/Johannes/2020_09_03/"
# mainFolder = "/data/Marjan/MB2020_lifs_2/"
mainFolder = "/data/Sandra/"

recordings = []
for cur,ds,fs in os.walk(mainFolder):
    #### if you wish to restrict to only certain folders: ####
    if "2019_08" not in cur: continue
    for f in fs:
        if not (f.endswith(".lif")):# or f.endswith(".nd2")):
            continue
        path = os.path.join(cur,f)
        recordings += [path]
recordings = sorted(recordings)[1:2]

In [7]:
status_orig = import_data(recordings, forceMetadataParse=False)

#################### /data/Sandra/2019/2019_08_09/Experiment27.lif



In [8]:
status = deepcopy(status_orig)
status.pxSize = status.pxSize.apply("{:.3f}".format).astype("str")
for c in status.columns:
    if "Size" in c and "px" not in c:
        status[c.replace("Size","")] = status[c]
        del status[c]

status["location"] = status.path.apply(lambda xi: os.path.split(xi)[0])
status["Freq"] = status["Frequency"].apply("{:.1f}".format).astype("str")
del status["Frequency"]
status["Duration"] = status["Duration"].apply(str).apply(lambda xi: xi.split()[-1].split(".")[0] )

firstCols = ["exp", "series","movie","Freq","Duration", "movie size [MB]", ]#,"images",]
status = status[[c for c in firstCols if c in status]+[c for c in status.columns if c not in firstCols]]
lastCols = ["path"]
status = status[[c for c in status.columns if c not in lastCols]+lastCols]
del status["pxUnit"]
del status["frame_range"]
try:
    status["movie size [MB]"] = status["movie size [MB]"].round(1)
except:
    pass
del status['gap']
status.pickles = status.pickles.apply(lambda xi: str(xi)[1:-1].replace(" ",""))

In [9]:
# status = status[~status[list("XYZT")].isna().all(1)]
# status = status[status.Z==1]

In [10]:
# status = status[~status[["movie","(re)do pickles"]].all(1)]

In [11]:
# status = status[~status.exp.apply(lambda xi: xi.endswith("nd2"))]

In [22]:
# status["line scan"] = status["line scan"].astype(int)
del status["line scan"]

In [23]:
# status = status[status_orig.Duration>pd.Timedelta("30s")]

In [32]:
nshow = len(status)
htmltable = generate_table(status.iloc[:nshow,:-2], max_rows=nshow)
ixOrder = OrderedDict([(f"table-{col}-{i}",{"col":col,"index":i}) \
                       for i in range(nshow) for col in ["movie","(re)do pickles"] if isinstance(status.loc[i,col],bool)])

app = JupyterDash(__name__, width=1000)

app.layout = html.Div([
    html.Div("Please check which of the following series you wish processed into movies/pickles. When done, please click on the button and follow instructions."),
    html.Br(),
    html.Button(id="save",children=["Prepare script"],n_clicks=0),
    html.Div(id="output"),
    html.Button(id="check-all",children=["Check all"],n_clicks=0),
    html.Div(htmltable,style={"width":"1000px"}),
    html.Pre(id="marked",children="------------",
             style={
                "width": "700px",
                "height": "300px",
                'overflowX': 'scroll',
                'overflowY': 'scroll',
                "display": "block" if "srdjan" in os.getcwd() else "none",
            }),
],style={"font-family":"Arial"})


@app.callback(
    Output("marked","children"),
    [Input(k,"value") for k in ixOrder]
)
def see(*manyinputs):
    print (manyinputs)
    out = deepcopy(ixOrder)
    for k,v in zip(out, manyinputs):
        out[k]["value"] = v
    out = list(out.values())
    
    for el in out:
        el["value"] = bool(len(el["value"]))
        i = status.index[el["index"]]
        el["rec"] = status.loc[i,"path"]
        el["ser"] = status.loc[i,"series"]
    out = pd.DataFrame(out)
    output = ""
    checklist_parse = []
    for (rec,ser),ddf in out.groupby(["rec","ser"]):
        el = {"rec":rec,"ser":ser}
        el["movie"] = ddf.query("col=='movie'")["value"].iloc[0]
        el["pickles"] = ddf.query("col=='(re)do pickles'")["value"].iloc[0]
        checklist_parse += [el]
    checklist_parse = pd.DataFrame(checklist_parse)
    #################################### good begin
    
    for rec in checklist_parse.rec.unique():
        outFile = "/.".join(os.path.split(rec))+".out"
        output += f"rm {outFile}\n"
    for _,row in checklist_parse.iterrows():
        if not row[["movie","pickles"]].any():
            continue
        outFile = "/.".join(os.path.split(row.rec))+".out"
        line = ""
        line += f"rm {outFile}\n"
        for _ in range(2):
            line += f'''echo "" >> {outFile}\n'''
        line += f'''echo "###### processing of {row.ser} started at: $(date)" >> {outFile}\n'''
#         line += f'''time notebooks/process_single_1.py --recording="{row.rec}" --series="{row.ser}" --verbose'''
        line += f'''time /data/useful_notebooks/process_single.py --recording="{row.rec}" --series="{row.ser}" --verbose'''
        if not row.movie:
            line += " --leave-movie"
        if not row.pickles:
            line += " --leave-pickles"
        line += f" >> {outFile}"
        output += line+"\n"
    #################################### good end
    
    return output

@app.callback(
    [Output(k,"value") for k in ixOrder],
    [Input("check-all","n_clicks")]
)
def checkall(nc):
    if nc>0:
        return tuple([[None] for k in  ixOrder])

@app.callback(
    Output("output","children"),
    [Input("save","n_clicks")],
    [State("marked","children")]
)
def prepare_script(n_clicks, text):
    if n_clicks>0:
        with open(os.path.expanduser("~/processing_script.sh"),"w") as f:
            f.write(text)
        return dcc.Markdown("""`processing_script.sh` prepared in your home folder.

Just open new terminal and run: `bash processing_script.sh` 

You can then close the window and wait until the processing is finished. 

_(For now, there is no reallyt good way how to follow progress, sorry. You can try the cell below...)_
""")

app._repr_html_() 
link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
HTML(f'open the following link in a different tab (do not close this tab!): <a href="{link2app}">{link2app}</a>')
# app

In [26]:
# Hacky way to check progress
for exp,df in status.groupby("exp"):
    printExp = True
    for i,row in df.iterrows():
        assert (status.loc[i].iloc[:2] == status_orig.loc[i,["exp","series"]]).all()
        images = [f".image_{'%i.%i'%fs if isinstance(fs,tuple) else fs}.png" for fs in status_orig.loc[i,"pickles"]]
        serDir = f"{row.path}_analysis/{row.series}"
        if os.path.isdir(serDir):
            imDone = sum([el in images for el in os.listdir(serDir)])
        else:
            imDone = 0
        outPutFile = row.path.replace(row.exp, "."+row.exp+".out")
        percDone = 100/len(images)*imDone
    #     if percDone>=100: continue
        print ("%20s %13s  %3i%%"%(row.exp if printExp else "", row.series, percDone))#, outPutFile, os.path.isfile(outPutFile))
    #     if os.path.isfile(outPutFile):
    #         output = open(outPutFile).read()
    #         last = output#-1]
    #         last = "\n".join(["\t"+l for l in last.splitlines()])
    #         last += "\n"
    #         print(last)

    #     break
        printExp = False
    print ("-"*50)

   Experiment47a.lif     Series053    0%
                         Series054  100%
--------------------------------------------------
   Experiment47b.lif     Series003  100%
                         Series004  100%
                         Series005  100%
                         Series006  100%
                         Series007  100%
--------------------------------------------------
   Experiment47c.lif     Series003  100%
                         Series004  100%
                         Series005  100%
                         Series011    0%
--------------------------------------------------
   Experiment47d.lif     Series003  100%
                         Series004  100%
                         Series005  100%
                         Series006  100%
                         Series007  100%
                         Series008    0%
--------------------------------------------------
   Experiment47e.lif     Series007  100%
                         Series008  100%
                 